Gera Difrações
=============
Gera um dado sintético de difrações

---

In [ ]:
from __future__ import print_function
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter
from scipy.ndimage.morphology import binary_closing

# Importa bibliotecas próprias
from utils import * 

# Configura o matplolib para plotar inline
%matplotlib qt5
#%matplotlib notebook

Define variáveis para construção das difrações

In [49]:
# Tamanho do dado. Dimensão final será um painel quadrado de data_size x data_size 
data_size = 750
# Intervalo de tempo entre as amostras (segundos)
dt = 0.004
# Intervalo de distância entre cada traço (metros)
dx = 12.5

---
Constrói modelo de velocidade

In [60]:
# Gera modelo de velocidades com distribuição normal em gradiente de velocidades no intervalo [v_begin, v_end]
v_begin = 1500.0
v_end = 3700.0
# Desvio padrão em relação ao gradiente
sigma_percent = 0.1

# Constrói modelo
mu = np.empty((data_size,data_size), dtype=np.float32)
mu[:,:] = np.arange(v_begin,v_end,(v_end-v_begin)/data_size).reshape((-1,1)) 
sigma = mu * sigma_percent
velocity_map = np.random.normal(loc=mu, scale=sigma)

In [61]:
# Aplica suavização

# Define o sigma do filtro gaussiano
sigma = 3.0

# Aplica filtro ao modelo normal
filtered_map = gaussian_filter(velocity_map, sigma=sigma)
vmin = min(velocity_map.min(), filtered_map.min())
vmax = max(velocity_map.max(), filtered_map.max())

title1='Modelo normal'
title2='Modelo suavizado'

multiple_event_viewer(velocity_map, filtered_map, dt=dt, 
                    vmin1=vmin, vmax1=vmax,
                    vmin2=vmin, vmax2=vmax,
                    title1=title1, title2=title2,
                    color1='rainbow', color2='rainbow')

Tamanho da imagem (em polegadas): 7.50 x 7.50
Coordenadas:  x - [0.0, 750.0], y - [0.0, 3.0]
Dimensão: (750, 750)



---
Constrói arestas das difrações e define amostras de ápices e não-ápices

In [52]:
# Constrói arestas das difrações

# Tamanho da perna das difrações. Cada difração terá tamanho final 2*diff_tail_size + 1
diff_tail_size = 80
# Borda do dado que não conterá ápice
border_size = 50
# Quantidade de difrações do dado
num_apices = 200

apices = np.random.randint(50, data_size-50, size=2*num_apices).reshape((-1,2))
mid_points = np.arange(data_size)*dx

diffrac_edges = np.zeros((data_size,data_size), dtype=np.float32)

for [t_ind, x_ind] in apices:
    t0 = t_ind*dt
    velocity = filtered_map[t_ind,x_ind]
    x_begin = max(0, x_ind-diff_tail_size)
    x_end = min(data_size, x_ind+diff_tail_size+1)
    horiz_indices = np.arange(x_begin,x_end).astype(int)
    offsets = mid_points[horiz_indices] - mid_points[x_ind]
    
    # Computa os tempos usando equação da curva
    times = np.sqrt(t0**2 + ((2*offsets)/velocity)**2)
    vert_indices = (np.floor(times/dt)).astype(int)
    # Considera somente os índices que não ultrapassaram o final do painel
    valid_indices = np.where(vert_indices < data_size)[0]
    # Cria curva de pseudo-refletividade o ápice tem valor 1.0 e decai até 0.0 usando função cosseno
    curve = np.ones(2*diff_tail_size+1, dtype=np.float32)
    curve = apply_taper(curve, diff_tail_size)
    # Pega o máximo entre a curva e o que já está no painel
    #curve = np.max([diffrac_edges[vert_indices[valid_indices],horiz_indices[valid_indices]].flatten(),
    #                curve[valid_indices]], axis=0)
    #diffrac_edges[vert_indices[valid_indices],horiz_indices[valid_indices]] = curve
    diffrac_edges[vert_indices[valid_indices],horiz_indices[valid_indices]] += curve[valid_indices]

In [53]:
# Visualiza arestas das difrações
view_seismic_stack(diffrac_edges, dt=dt)

Tamanho da imagem (em polegadas): 7.50 x 7.50


---
Convolve dado de arestas com wavelet

In [54]:
# Constrói Wavelet 2D
frequency = 20.0
taper_size = 4
window_size = 71

wavelet = ricker_wavelet(frequency, window_size, dt)
wavelet = wavelet.reshape((1,-1)).T
wavelet_2D = np.empty((window_size, taper_size*2+1), dtype=np.float32)
wavelet_2D[:,:] = wavelet
# Suaviza bordas aplicando função de taper
if taper_size > 0:
    wavelet_2D = apply_taper(wavelet_2D, taper_size)
print("Wavelet_2D", wavelet_2D.shape)

# Visualiza
color='seismic'
resize=10.0

view_seismic_stack(wavelet_2D, resize=resize, color=color, dt=dt, vmin=-wavelet_2D.max(), vmax=wavelet_2D.max())

Wavelet_2D (71, 9)
Tamanho da imagem (em polegadas): 0.90 x 7.10


In [55]:
# Convolve com difrações
diffractions = convolve2d(diffrac_edges, wavelet_2D, mode='same')

In [56]:
# Define pontos de não-ápices balanceado entre cauda e fundo

apex_radius = 8
background_threshold = diffractions.max()*0.007
num_tail_points = num_apices/2
num_background_points = num_apices/2

# Sorteia pontos de cauda
curve = np.ones(2*diff_tail_size+1, dtype=np.float32)
curve = apply_taper(curve, diff_tail_size)
threshold = curve[diff_tail_size+apex_radius]
tail_indices = np.where(np.logical_and(diffrac_edges>0.0, diffrac_edges<threshold))
tail_choice = np.random.choice(tail_indices[0].size, size=num_tail_points, replace=False)
tail_samples = zip(tail_indices[0][tail_choice],tail_indices[1][tail_choice])

# Sorteia pontos de fundo
# Transforma o dado em binário
abs_diff = np.abs(diffractions)
abs_diff[abs_diff < background_threshold] = 0.0
abs_diff[abs_diff >= background_threshold] = 1.0
# Faz fechamento morfológico para remover pequenos buracos
structure = np.ones((2,2))
abs_diff = binary_closing(abs_diff, structure=structure)
# Desconsidera bordas
abs_diff[0:border_size,:] = 1.0
abs_diff[-border_size:,:] = 1.0
abs_diff[:,0:border_size] = 1.0
abs_diff[:,-border_size:] = 1.0
# Sorteia pontos de fundo
background_indices = np.where(abs_diff==0.0)
background_choice = np.random.choice(background_indices[0].size, size=num_background_points, replace=False)
background_samples = zip(background_indices[0][background_choice],background_indices[1][background_choice])

# Converte índices para picks no formato (tempo, cdp)
apex_picks = index_to_picks(apices, 0, dt)
tail_picks = index_to_picks(tail_samples, 0, dt)
background_picks = index_to_picks(background_samples, 0, dt)

print("Picks de ápices:", apex_picks.shape)
print("Picks de caudas:", tail_picks.shape)
print("Picks de fundo:", background_picks.shape)

Picks de ápices: (200, 2)
Picks de caudas: (100, 2)
Picks de fundo: (100, 2)


In [ ]:
# Visualiza região de fundo

view_seismic_stack(abs_diff, vmin=0.0, vmax=1.0, dt=dt, nonapices=background_picks)

In [57]:
# Visualiza dado convolvido e pontos anotados
color='gray'
clip=1.0

vmax = diffractions.max()


view_seismic_stack(diffractions, apices=apex_picks, intersects=tail_picks, nonapices=background_picks,
                   color=color, dt=dt, clip_percent=clip, vmin=-vmax, vmax=vmax)

Tamanho da imagem (em polegadas): 7.50 x 7.50


In [ ]:
# Visualiza dado convolvido e pontos anotados
color='gray'
clip=1.0

vmax = diffractions.max()
vmin = -vmax

view_seismic_stack(diffractions, color=color, dt=dt, clip_percent=clip, vmin=vmin, vmax=vmax)

In [58]:
# Visualiza dado convolvido e arestas
color='gray'
clip=1.0

multiple_event_viewer(diffractions, diffrac_edges, dt=dt, clip_percent1=clip, clip_percent2=clip,
                     vmin1=-diffractions.max(), vmax1=diffractions.max(),
                     vmin2=0.0, vmax2=diffrac_edges.max(),
                     color1=color, color2=color)

Tamanho da imagem (em polegadas): 7.50 x 7.50
Coordenadas:  x - [0.0, 750.0], y - [0.0, 3.0]
Dimensão: (750, 750)

evento=1.0, cdp=219.816105, tempo=1.305024
evento=1.0, cdp=445.484862, tempo=2.286148


---
Salva arquivos de _picks_ com ápices e não-ápices e arquivo com difrações

In [ ]:
# Salva picks

apex_file = 'labels/apices.txt'
noapex_file = 'labels/no_apices.txt'

# Salva ápices
save_picks(apex_file, apex_picks)

# Salva não-ápices
noapex_picks = np.concatenate((tail_picks, background_picks))
save_picks(noapex_file, noapex_picks)

In [ ]:
# Salva difrações
diff_file = 'difracoes_sinteticas/diff.pickle'

pickle_data(diff_file, [diffractions], ['diffractions'])

In [ ]:
data = load_pickle(diff_file)
diffs = data['diffractions']
print(diffs.shape)